dataset import

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
X=np.load("path to dataset features (X_c1.np)")
Y=np.load("path to dataset labels (Y_c1.np")
X=X/255.0
plt.imshow(X[706])
print(Y[706])
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42,stratify=Y)
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
#print(Y_test)
a=[0,0,0,0,0,0,0,0,0,0]
for i in range(Y_test.shape[0]):
  a[int(Y_test[i])]=a[int(Y_test[i])]+1
print(a)

Model Architecture

In [ ]:
import tensorflow as tf
from tensorflow import Tensor
from tensorflow import keras
from tensorflow.python.keras.layers import Input, Conv2D, ReLU, BatchNormalization,\
                                    Add, AveragePooling2D, Flatten, Dense
from tensorflow.python.keras.models import Model

initializer = tf.keras.initializers.HeNormal()

def scheduler(epoch, lr):
  if (epoch%15)==0:
    return lr * tf.math.exp(-0.1)
  else:
    return lr

def relu_bn(inputs: Tensor) -> Tensor:
    relu = ReLU()(inputs)
    bn = BatchNormalization()(relu)
    return bn

def relu_u(inputs: Tensor) -> Tensor:
    relu = ReLU()(inputs)
    #bn = BatchNormalization()(relu)
    return relu

def bottleneck(x: Tensor, kernels:int, dilation: int) -> Tensor:
    
   
    y = Conv2D(kernel_size=1,
               strides= 1,
               filters=int(kernels/4),
               padding="same",kernel_initializer=initializer)(x)
    y = relu_bn(y)
    
    y = Conv2D(kernel_size=3,
               strides=1,
               filters=int(kernels/4),
               padding="same",kernel_initializer=initializer)(y)
    y = relu_bn(y)
    
    y = Conv2D(kernel_size=1,
               strides= 1,
               filters=kernels,
               padding="same",kernel_initializer=initializer)(y)
    y = relu_bn(y)
    
    out = Add()([x, y])
    
    y1 = Conv2D(kernel_size=1,
               strides= 1,
               filters=int(kernels/4),
               padding="same",kernel_initializer=initializer)(out)
    y1 = relu_bn(y1)
    
    y1 = Conv2D(kernel_size=3,
               strides=1,
               filters=int(kernels/4),
               dilation_rate=dilation,
               padding="same",kernel_initializer=initializer)(y1)
    y1 = relu_bn(y1)
    
    y1 = Conv2D(kernel_size=1,
               strides= 1,
               filters=kernels,
               padding="same",kernel_initializer=initializer)(y1)
    y1 = relu_bn(y1)
    
    out1 = Add()([out, y1])   

    return out1

def create_net():
    
    inputs = Input(shape=(256, 256, 3))
    
    t = Conv2D(kernel_size=3,
               strides=2,
               filters=64,
               padding="valid",kernel_initializer=initializer)(inputs)
    t = relu_bn(t)
    
    t = Conv2D(kernel_size=3,
               strides=2,
               filters=128,
               padding="valid",kernel_initializer=initializer)(t)
    t = relu_bn(t)
    
    t = bottleneck(t,kernels=128,dilation=2)
    
    t = Conv2D(kernel_size=3,
               strides=2,
               filters=256,
               padding="valid",kernel_initializer=initializer)(t)
    t = relu_bn(t)
    
    t = bottleneck(t,kernels=256,dilation=4)
    
    
    t = Conv2D(kernel_size=3,
               strides=2,
               filters=128,
               padding="valid",kernel_initializer=initializer)(t)
    t = relu_bn(t)
    
    t = bottleneck(t,kernels=128,dilation=8)
    
    t = Conv2D(kernel_size=3,
               strides=2,
               filters=64,
               padding="valid",kernel_initializer=initializer)(t)
    t = relu_bn(t)
    
    t=bottleneck(t,kernels=64,dilation=4)

    t = Conv2D(kernel_size=3,
               strides=2,
               filters=16,
               padding="valid",kernel_initializer=initializer)(t)
    t = relu_bn(t)


    '''
    t=bottleneck(t,kernels=32,dilation=2)

    t = Conv2D(kernel_size=3,
               strides=2,
               filters=16,
               padding="valid")(t)
    t = relu_bn(t)
'''
    t = Flatten()(t)
    outputs = Dense(10, activation='softmax')(t)
    
    model = Model(inputs, outputs)
    
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.005),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model


Model Summary

In [ ]:
net = create_net()
print(net.summary())

Training using keras .fit method

In [ ]:
net = create_net()
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)
a=net.fit(X_train,Y_train,batch_size=8,epochs=40,validation_split=0.1,shuffle=True,callbacks=[callback])

Testing the trained model

In [ ]:
print("Evaluate on test data")
results = net.evaluate(X_test, Y_test, batch_size=1)
print("test loss, test acc:", results)

Evaluating the model accuracy and confusion matrix

In [ ]:
import pandas as pd
import seaborn as sns
y_=net.predict(X_test)
y_pred=np.argmax(tf.nn.softmax(y_),axis=1)

con_mat = tf.math.confusion_matrix(labels=Y_test, predictions=y_pred).numpy()

con_mat_norm = np.around(con_mat.astype('float') / con_mat.sum(axis=1)[:, np.newaxis], decimals=2)
classes=['0','1','2','3','4','5','6','7','8','9']
con_mat_df = pd.DataFrame(con_mat_norm,
                     index = classes, 
                     columns = classes)


figure = plt.figure(figsize=(8, 8))
sns.heatmap(con_mat_df, annot=True,cmap=plt.cm.Blues)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

from sklearn.metrics import classification_report
print(classification_report(Y_test, y_pred))
from sklearn.metrics import accuracy_score
accuracy_score(Y_test, y_pred)

Save the model in  .h5 file

In [ ]:
net.save('path to save model in .h5 format')

Converting tensorflow model to tensorflow lite model

In [ ]:
import tensorflow as tf
from tensorflow import lite
from tensorflow import keras
mode = keras.models.load_model('path to model .h5 file')
#mode.save('/gdrive/My Drive/final_model_.h5')
converter = lite.TFLiteConverter.from_keras_model(mode)
tflite_model = converter.convert()

open('provide file name and path to store tflite model', 'wb').write(tflite_model)